In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

# Both MNIST and Fashion-MNIST can be loaded from Keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist

from tqdm import tqdm
from tqdm.autonotebook import tqdm, trange

In [2]:
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Convert image pixels to floats between 0 and 1
X_train = x_train / 255
X_test = x_test / 255

# Convert output to one hot encoding
Y_train = to_categorical(y_train, 10) 
Y_test = to_categorical(y_test, 10)

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

In [3]:
import tensorflow as tf

gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(layers.Input(shape = (28, 28, 1,)))
model.add(layers.Conv2D(32, kernel_size = (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D( pool_size = (2, 2)))
model.add(layers.Conv2D(64, kernel_size = (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D(pool_size = (2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation = "softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print( model.summary() )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                16010     
Total params: 34,826
Trainable params: 34,826
Non-trainable params: 0
____________________________________________________

In [4]:
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)

batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size, drop_remainder=True)

In [5]:
train_data = list(train_dataset)

In [6]:
train_acc_metric = keras.metrics.CategoricalAccuracy()

In [7]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

In [8]:
epochs = 50

for epoch in range(epochs):    
    steps = trange(len(train_data), bar_format="{desc}\t{percentage:3.0f}% {r_bar}")
    for i in steps:
        step = i
        x_batch_train = train_data[i][0]
        y_batch_train = train_data[i][1]
        
        loss_value = train_step(x_batch_train, y_batch_train)
        steps.set_description("Epoch " + str(epoch+1) + '/' + str(epochs) + "\tLoss: " + str(float(loss_value))[:6]
                              + "\tAccuracy: " + str(float(train_acc_metric.result()))[:6])


Epoch 1/50	Loss: 1.6948	Accuracy: 0.7398: 	100% | 937/937 [00:03<00:00, 269.43it/s]
Epoch 2/50	Loss: 1.6292	Accuracy: 0.7932: 	100% | 937/937 [00:02<00:00, 419.61it/s]
Epoch 3/50	Loss: 1.6200	Accuracy: 0.8163: 	100% | 937/937 [00:02<00:00, 412.48it/s]
Epoch 4/50	Loss: 1.6009	Accuracy: 0.8297: 	100% | 937/937 [00:02<00:00, 424.98it/s]
Epoch 5/50	Loss: 1.5833	Accuracy: 0.8389: 	100% | 937/937 [00:02<00:00, 412.39it/s]
Epoch 6/50	Loss: 1.5682	Accuracy: 0.8462: 	100% | 937/937 [00:02<00:00, 420.93it/s]
Epoch 7/50	Loss: 1.5744	Accuracy: 0.8523: 	100% | 937/937 [00:02<00:00, 419.47it/s]
Epoch 8/50	Loss: 1.5487	Accuracy: 0.8574: 	100% | 937/937 [00:02<00:00, 423.37it/s]
Epoch 9/50	Loss: 1.5591	Accuracy: 0.8619: 	100% | 937/937 [00:02<00:00, 424.16it/s]
Epoch 10/50	Loss: 1.5506	Accuracy: 0.8657: 	100% | 937/937 [00:02<00:00, 413.98it/s]
Epoch 11/50	Loss: 1.5384	Accuracy: 0.8691: 	100% | 937/937 [00:02<00:00, 419.64it/s]
Epoch 12/50	Loss: 1.5406	Accuracy: 0.8722: 	100% | 937/937 [00:02<00:00, 4